In [1]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\duany\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\treebank.zip.


True

## Get all the parsed sentences as trees from PTB

In [2]:
from nltk.corpus import treebank

In [8]:
treebank.fileids()

['wsj_0001.mrg',
 'wsj_0002.mrg',
 'wsj_0003.mrg',
 'wsj_0004.mrg',
 'wsj_0005.mrg',
 'wsj_0006.mrg',
 'wsj_0007.mrg',
 'wsj_0008.mrg',
 'wsj_0009.mrg',
 'wsj_0010.mrg',
 'wsj_0011.mrg',
 'wsj_0012.mrg',
 'wsj_0013.mrg',
 'wsj_0014.mrg',
 'wsj_0015.mrg',
 'wsj_0016.mrg',
 'wsj_0017.mrg',
 'wsj_0018.mrg',
 'wsj_0019.mrg',
 'wsj_0020.mrg',
 'wsj_0021.mrg',
 'wsj_0022.mrg',
 'wsj_0023.mrg',
 'wsj_0024.mrg',
 'wsj_0025.mrg',
 'wsj_0026.mrg',
 'wsj_0027.mrg',
 'wsj_0028.mrg',
 'wsj_0029.mrg',
 'wsj_0030.mrg',
 'wsj_0031.mrg',
 'wsj_0032.mrg',
 'wsj_0033.mrg',
 'wsj_0034.mrg',
 'wsj_0035.mrg',
 'wsj_0036.mrg',
 'wsj_0037.mrg',
 'wsj_0038.mrg',
 'wsj_0039.mrg',
 'wsj_0040.mrg',
 'wsj_0041.mrg',
 'wsj_0042.mrg',
 'wsj_0043.mrg',
 'wsj_0044.mrg',
 'wsj_0045.mrg',
 'wsj_0046.mrg',
 'wsj_0047.mrg',
 'wsj_0048.mrg',
 'wsj_0049.mrg',
 'wsj_0050.mrg',
 'wsj_0051.mrg',
 'wsj_0052.mrg',
 'wsj_0053.mrg',
 'wsj_0054.mrg',
 'wsj_0055.mrg',
 'wsj_0056.mrg',
 'wsj_0057.mrg',
 'wsj_0058.mrg',
 'wsj_0059.mrg

In [11]:
tree_list = []
for idx in range(1, 199):
    file = 'wsj_' + str(idx).zfill(4) + '.mrg'
    for tree in treebank.parsed_sents(file):
        tree_list.append(tree)

In [12]:
len(tree_list)

3911

In [13]:
type(tree_list[0])

nltk.tree.tree.Tree

## Extract a CFG Grammar Rule Set from the Ground Truth Trees

In [16]:
from nltk.tree import Tree

def get_counts(tree_list):
    # The input is treebank, a list of NLTK trees
    # You will return 
    #  - NTs, a list of unique nonterminals
    #  - count_NT, a dictionary mapping NTs to their counts
    #  - count_rule, a dictionary mapping rules (represented as nested tuples ("S", ("NP", "VP")) ) to their counts
    
    NTs = []
    count_NT = {}
    count_rule = {}
    
    # Your implementation here
    queue = []
    for tree in tree_list:
        queue.append(tree)

        # DFS Traverse for Each Tree
        while len(queue)>0:
            current_node = queue.pop()
        
            lhs_label = current_node.label()
            rhs_labels = []
            for i in range(len(current_node)):
                if type(current_node[i]) == Tree:
                    rhs_labels.append(current_node[i].label())
                else:
                    rhs_labels.append(current_node[i])
            children = [current_node[i] for i in range(len(current_node))]

            if lhs_label not in NTs:
                NTs.append(lhs_label)
        
            if lhs_label not in count_NT.keys():
                count_NT[lhs_label] = 1
            else:
                count_NT[lhs_label] += 1
        
            rule = (lhs_label, tuple(rhs_labels))
            if rule not in count_rule.keys():
                count_rule[rule] = 1
            else:
                count_rule[rule] += 1
        
            for node in children:
                if type(node)==Tree:
                    queue.append(node)


    # hint: keep a queue of nodes you need to visit
   
    ####################
    
    return NTs, count_NT, count_rule

In [17]:
NTs, count_NT, count_rule = get_counts(tree_list)

In [23]:
def build_pcfg(NTs, count_NT, count_rule):
    # inputs are as described above
    # the output is the grammar dictionary, which will map a tuple representing the 
    # right-hand side of a rule (rhs) to a list of pairs, the first 
    # entry being the rules probability, and the second being the nonterminal on
    # the left-hand side of the rule (lhs)
    # that is, grammar[("NP", "VP")]  = [(1.0, "S")]  
    #          grammar[("V", "NP")] = [(0.5, "VP")]
    # (these are examples, not real numbers!)
       
    grammar = {}
    
    # Your implementation here
    for rule in count_rule.keys():
        lhs_label = rule[0]
        prob = count_rule[rule]/count_NT[lhs_label]
        if lhs_label in grammar.keys():
            grammar[lhs_label].append((rule[1], prob))
        else:
            grammar[lhs_label] = [(rule[1], prob)]
    ####################
    
    return grammar

In [24]:
PCFG_PTB = build_pcfg(NTs, count_NT, count_rule)

In [25]:
PCFG_PTB

{'S': [(('NP-SBJ', 'VP', '.'), 0.1623278954066875),
  (('NP-SBJ-1', 'VP', '.'), 0.03066065023718616),
  (('NP-SBJ', 'NP-PRD'), 0.005437926645840564),
  (('S-TPC-1', ',', 'NP-SBJ', 'VP', '.'), 0.007289135716765012),
  (('-NONE-',), 0.055189170426935094),
  (('S-TPC-2', ',', 'NP-SBJ', 'VP', '.'), 0.003702418141848895),
  (('NP-SBJ', 'VP'), 0.3919935207682518),
  (('SBAR-ADV', ',', 'NP-SBJ', 'VP', '.'), 0.004975124378109453),
  (('NP-SBJ-2', 'VP'), 0.023255813953488372),
  (('NP-SBJ', 'VP', '.', "''"), 0.007751937984496124),
  (('NP-SBJ-1', 'VP'), 0.04280920976512785),
  (('PP-LOC', ',', 'NP-SBJ', 'VP', '.'), 0.007867638551428902),
  (('NP-SBJ-4', 'VP'), 0.0018512090709244474),
  (('NP-SBJ-5', 'VP'), 0.00046280226773111186),
  (('PP-TMP', ',', 'NP-SBJ', 'VP', '.'), 0.007520536850630568),
  (('PP-TMP', ',', 'NP-SBJ-6', 'VP', '.'), 0.00011570056693277797),
  (('NP-SBJ-7', 'VP', '.'), 0.00011570056693277797),
  (('NP-SBJ-8', 'VP'), 0.00023140113386555593),
  (('``', 'CC', 'NP-SBJ-1', 'VP', '

In [26]:
lines = ['hey you', 'hey world', 'bye !']

with open('./your_file.txt', 'w') as f:
    for line in lines:
        f.write(f"{line}\n")